In [40]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.coordinates import SkyCoord

from dotenv import load_dotenv
load_dotenv(override=True)

ROOT_PATH = os.environ.get('ROOT_PATH')
if not ROOT_PATH in sys.path: sys.path.append(ROOT_PATH)

from main_code.utils.filepaths import H22_FILEPATH
from main_code.utils.constants import LIGHTSPEED

In [ ]:
# Load Cullan's data
df_h22 = pd.read_csv(H22_FILEPATH)[["objid", "zcmb_group"]]
df_h22["objid"] = df_h22["objid"].apply(lambda x: "SDSS" + str(x))

# Load my SDSS data
df = pd.read_csv("../../data/preprocessed/spectrophoto/sdss.csv")[["tmass", "objid"]]
df["objid"] = df["objid"].apply(lambda x: "SDSS" + str(x))

# Merge
df = df.merge(df_h22, on="objid", how="inner").drop("objid", axis=1)

# Calculate z_dist_est

In [ ]:
survey = "SDSS"

# Load data
df = pd.read_csv(f"../../data/preprocessed/spectrophoto/{survey.lower()}.csv")

# # Load Cullan's data
# df_h22 = pd.read_csv(H22_FILEPATH)[["objid", "zcmb_group"]]
# df_h22["objid"] = df_h22["objid"].apply(lambda x: "SDSS" + str(x))

# # Load my SDSS data
# df = pd.read_csv("../../data/preprocessed/spectrophoto/sdss.csv")[["tmass", "objid"]]
# df["objid"] = df["objid"].apply(lambda x: "SDSS" + str(x))

# # Merge
# df_h22 = df.merge(df_h22, on="objid", how="inner").drop("objid", axis=1)

# Load Cullan's data
df_h22 = pd.read_csv(H22_FILEPATH)[["objid", "zcmb_group"]]

if survey == "SDSS":
    # Get Cullan's CMB group redshift directly
    df = df.merge(df_h22, on="objid", how="left")

    df["objid"] = df["objid"].apply(lambda x: "SDSS" + str(x))
    df["z_dist_est"] = df["zcmb_group"]
    df['Group'] = np.where(df['tempel_counterpart'] == True, df['Group'], -1)
    df['Nr'] = np.where(df['tempel_counterpart'] == True, df['Nr'], 1)
elif survey == "LAMOST":
    # Load SDSS data, combine with H22
    df_sdss = pd.read_csv("../../data/preprocessed/spectrophoto/sdss.csv")[["tmass", "objid"]]
    df_sdss["objid"] = df_sdss["objid"].apply(lambda x: "SDSS" + str(x))
    df_ = df_sdss.merge(df_h22, on="objid", how="left")

    # Use Cullan's CMB group redshift if available, use Tempel's if not
    df = df.merge(df_, on="tmass", how="left")

    # TODO: REMOVE
    df["z_cmb"] = df["z_lamost"]

    df['z_dist_est'] = np.where(~df['zcmb_group'].isna(), df['zcmb_group'], np.where(df["tempel_counterpart"] == True, df["zcl"], df["z_cmb"]))

    
# if survey in ['SDSS', 'LAMOST']:
#     df['z_dist_est'] = np.where(df['tempel_counterpart'] == True, df['zcl'], df['z_cmb'])
#     df['Group'] = np.where(df['tempel_counterpart'] == True, df['Group'], -1)
#     df['Nr'] = np.where(df['tempel_counterpart'] == True, df['Nr'], 1)
# else:
#     df['z_dist_est'] = np.where(df['cz_gr'] != 0., df['cz_gr'] / LIGHTSPEED, df['z_cmb'])

# return df

# df
# df[df["Group"] == 57543]

In [62]:
df_h22

,tmass,zcmb_group
0,2MASXJ14213079+1757463,0.05345
1,2MASXJ10193574+4240552,0.07650
2,2MASXJ08262749+5439122,0.06864
3,2MASXJ13120148+2549521,0.06504
4,2MASXJ11142839+4133276,0.07335
...,...,...
31537,2MASXJ11464207+2507350,0.08859
31538,2MASXJ10303464+4109486,0.09289
31539,2MASXJ10363843+1410175,0.01122
31540,2MASXJ12011024+2240162,0.08518
